In [6]:
from pyspark import SparkContext,HiveContext
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import datetime
from pyspark.sql.window import Window

print ("Lets start")

Lets start


In [1]:
'''
1.Reading the input file from hdfs
2.Removing the header record

'''
teamRdd = sc.textFile("/user/data/fujitsu/Team.txt").map(lambda x : x.split("\t")).filter(lambda x : x[0] != 'id')
teamRdd.take(5)

[[u'1', u'9987', u'673', u'KRC Genk', u'GEN'],
 [u'2', u'9993', u'675', u'Beerschot AC', u'BAC'],
 [u'3', u'10000', u'15005', u'SV Zulte-Waregem', u'ZUL'],
 [u'4', u'9994', u'2007', u'Sporting Lokeren', u'LOK'],
 [u'5', u'9984', u'1750', u'KSV Cercle Brugge', u'CEB']]

In [7]:
def buildSchemaTeam():
    print("Schema Creation Started")
    schema = StructType(
        [
            StructField("id", StringType(),True),
            StructField("team_api_id" ,StringType(),True),
            StructField("team_fifa_api_id",StringType(),True),
            StructField("team_long_name", StringType(),True),
            StructField("team_short_name", StringType(),True)   
            
        ]
    )
    print("Schema Creation Completed")
    return schema

In [8]:
df = sqlContext.createDataFrame(teamRdd,schema =buildSchemaTeam() )
df.show()

Schema Creation Started
Schema Creation Completed
+----+-----------+----------------+--------------------+---------------+
|  id|team_api_id|team_fifa_api_id|      team_long_name|team_short_name|
+----+-----------+----------------+--------------------+---------------+
|   1|       9987|             673|            KRC Genk|            GEN|
|   2|       9993|             675|        Beerschot AC|            BAC|
|   3|      10000|           15005|    SV Zulte-Waregem|            ZUL|
|   4|       9994|            2007|    Sporting Lokeren|            LOK|
|   5|       9984|            1750|   KSV Cercle Brugge|            CEB|
|   6|       8635|             229|      RSC Anderlecht|            AND|
|   7|       9991|             674|            KAA Gent|            GEN|
|   8|       9998|            1747|           RAEC Mons|            MON|
|   9|       7947|                |       FCV Dender EH|            DEN|
|  10|       9985|             232|   Standard de Li�ge|            STL|
|

In [9]:
sqlContext.sql("use fujitsu")
sqlContext.sql(''' create table IF NOT EXISTS team ( id int,team_api_id int,team_fifa_api_id int,team_long_name varchar(50),
team_short_name varchar(3)) stored as parquet''')
sqlContext.sql(" select * from fujitsu.team ").show()

+---+-----------+----------------+--------------+---------------+
| id|team_api_id|team_fifa_api_id|team_long_name|team_short_name|
+---+-----------+----------------+--------------+---------------+
+---+-----------+----------------+--------------+---------------+



In [10]:
df.write.insertInto('fujitsu.team')

In [11]:
sqlContext.sql(" select * from fujitsu.team limit 10 ").show()

+---+-----------+----------------+-----------------+---------------+
| id|team_api_id|team_fifa_api_id|   team_long_name|team_short_name|
+---+-----------+----------------+-----------------+---------------+
|  1|       9987|             673|         KRC Genk|            GEN|
|  2|       9993|             675|     Beerschot AC|            BAC|
|  3|      10000|           15005| SV Zulte-Waregem|            ZUL|
|  4|       9994|            2007| Sporting Lokeren|            LOK|
|  5|       9984|            1750|KSV Cercle Brugge|            CEB|
|  6|       8635|             229|   RSC Anderlecht|            AND|
|  7|       9991|             674|         KAA Gent|            GEN|
|  8|       9998|            1747|        RAEC Mons|            MON|
|  9|       7947|            null|    FCV Dender EH|            DEN|
| 10|       9985|             232|Standard de Li�ge|            STL|
+---+-----------+----------------+-----------------+---------------+

